In [1]:
%load_ext autoreload
%autoreload 2
from sys import path
path.append('../')

In [2]:
import re

import seaborn as sb
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [3]:
from utils.loading import load_data
from utils.protocol_creation import summarize_alliance
from utils.ai_summary import create_chatgpt_prompt
from utils.parser import parse_event, find_all_patterns

In [102]:
data = load_data('../data')

/home/tobi/development/start_hack_2023/notebooks/../utils/loading.py:16: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  messages2 = pd.read_csv(path /'chat_messages_2_fixed.csv')


In [103]:
output = pd.DataFrame(data.alliance_membership.sort_values('timestamp').groupby('alliance_id')['size'].last())
output.rename(columns={'size':'current_size'}, inplace=True)

In [104]:
messages = data.messages.loc[:, ['account_id', 'alliance_id', 'raw_message', 'risk', 'timestamp']]
messages.rename(columns={'account_id':'author', 'raw_message':'content'}, inplace=True)
messages['isAction'] = False
actions = data.alliance_membership.loc[:, ['alliance_id', 'account_id', 'action', 'timestamp']]
actions['risk']=-1
actions['isAction'] = True
actions.rename(columns={'account_id':'author', 'action':'content'}, inplace=True)
protocol = pd.concat([messages, actions], ignore_index=True).sort_values('timestamp', ascending=True)
protocol = protocol.groupby('alliance_id').apply(lambda x: x.to_dict(orient='index'))
output['protocol'] = protocol 

In [175]:
accounts = data.accounts.copy()
accounts.session_duration_today /= 3600
accounts.session_duration /= 3600
accounts['time_existent'] = (accounts.last_active_date - accounts.created_time)
accounts.set_index(['alliance_id','account_id'], inplace=True)
account_aliance_groups = data.messages.groupby(['alliance_id', 'account_id'])
account_means = account_aliance_groups[["risk", 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT', 'EXTREMISM',
                                        'SUBVERSIVE', 'SENTIMENT', 'POLITICS']].mean()
account_means['filtered_total'] = account_aliance_groups['filterted'].sum()
#account_means = pd.concat([account_means, accounts.loc[:,['revenue_today', 'revenue', 'session_duration_today', 'session_duration', 'level', 'time_existent']]],axis=1)
accounts.loc[:,['revenue_today', 'revenue', 'session_duration_today', 'session_duration', 'level', 'time_existent']]
#account_means.reset_index('alliance_id', drop=False, inplace=True)
#account_means = pd.concat([account_means, accounts], axis=1)
# def enrich(x):
#     x.loc[:,['revenue_today', 'revenue', 'session_duration_today', 'session_duration', 'level', 'time_existent']] = \
#     accounts.loc[:,['revenue_today', 'revenue', 'session_duration_today', 'session_duration', 'level', 'time_existent']]
#     return x.reset_index('alliance_id', drop=True).to_json(orient='index')
#for i,g in account_means.groupby('alliance_id'):
#    output.loc[i,'alliance_account_summaries'] = g.to_dict(orient='index')
#output['alliance_account_summaries'] = account_means.reset_index('alliance_id', drop=False).groupby('alliance_id').apply(lambda x: x.to_dict(orient='index'))

,,revenue_today,revenue,session_duration_today,session_duration,level,time_existent
alliance_id,account_id,,,,,,
7f87a7d72c4038b8845735c772a0407cb58d4a443508d1e6eafb1478c5bda255,e10cd6ddf379b98c52e59529026359ad80e53ebecd38cd69773a26a2a0d2e3e6,0.0,0.00,1.465398,332.623748,33.0,243 days 14:02:46.024000
beb86b8e68696fb10c3b3c55898a495568579cc17e2eb4b7e96f20c8cddaad1e,592103594bc6e660e1d0caa9e046d7b7f6da3eede4e893d88080875d1d40ab3b,0.0,0.00,0.469944,114.967712,31.0,459 days 09:39:27.284000
61b3a23ded0bdd6d3df3d0d7f741af2eb48e6f9bc662dcfaebdcd0c9db68a9f4,7be671feb8066897a6292e03e520aa4ffbc9cae00ffe995a52b41a7e6b922613,0.0,0.00,1.164711,155.741171,32.0,608 days 14:37:47.358000
a4979799f92bcec14255788a66d24be68de82c8afa125239124814c3ddc72359,0cea88f4366301f6c58f45d131180aeb6108df7820f3ccc7f496bb757ed9c6e7,0.0,0.00,1.480901,362.051459,38.0,557 days 17:19:58.304000
546c74e081f1d793ad223ac6ac4cedf226b0a4d285fae2b7746180317ab6147b,57eb9194ccc208704bf991dd56a5cb3fff63c9b0ce61f6a1b22f37f2f587fe23,0.0,0.00,1.097063,406.056463,35.0,278 days 10:49:45.776000
...,...,...,...,...,...,...,...
b981c6e1505f98aece038f634159bcb88fe4165a6f8436adfbf9966f4743997e,0b3dc5e1e27efe270d4efc35d0c45431ea776c88dc0765f732711c408a7695dd,0.0,9.98,0.220889,263.998971,29.0,1822 days 20:39:55.975000
569c41f38d9d1fd8a421615b54edc58ad09d4ecc554da94f4243283e40e2c2a8,514ec922ee48e22eb15084fab3cbf88f817e4030edec051f869b8b51d3a63297,0.0,0.99,0.875148,1510.549648,50.0,1535 days 04:30:58.149000
6f09682eb5d3c6bbe3a7bee1f35e594da10f056526036250a7edbf22ba37a170,52837ef4e11812234cf93ead2d84eef2ce7cd881e915a4b40ed9a4e7a37a3d56,0.0,21.94,4.671309,270.204995,35.0,1337 days 10:06:56.332000


In [180]:
res = account_means.join(accounts.loc[:,['revenue_today', 'revenue', 'session_duration_today', 'session_duration', 'level', 'time_existent']], on='account_id')
                         
                         

ValueError: len(left_on) must equal the number of levels in the index of "right"

In [158]:
account_means.level.isna().sum()

7929

In [125]:
data.accounts[data.accounts.account_id=='9a1c2694a954d8da24b59dfe122389d85f75dbbe3aa81b50be207e91d67772a6']

,account_id,alliance_id,session_count,session_duration,transaction_count,revenue,date,active,account_state,last_active_date,level,created_language,created_country_code,created_time,session_count_today,session_duration_today,transaction_count_today,revenue_today,last_login_game_client_language,last_login_country_code
91651,9a1c2694a954d8da24b59dfe122389d85f75dbbe3aa81b...,4b391891cb5779b23b64ca688641f72f72b57836f49c31...,12889,6.183709e+06,14,35.86,2023-03-01,True,3,2023-03-01,50.0,en,AE,2016-03-05 19:08:28.210,8,5683.582,0,0.0,en-AE,AE


In [106]:
data.messages.filtered = data.messages.filtered.astype('float')
alliance_summary = data.messages.groupby('alliance_id')[["risk", 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT', 'EXTREMISM',
                                         'SUBVERSIVE', 'SENTIMENT', 'POLITICS', 'filtered']].mean()
output.loc[:,["risk", 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT', 'EXTREMISM', 'SUBVERSIVE', 'SENTIMENT', 'POLITICS', 'filtered']] =\
alliance_summary.loc[:,["risk", 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT',
                        'EXTREMISM', 'SUBVERSIVE', 'SENTIMENT', 'POLITICS', 'filtered']]

In [46]:
found_patterns = find_all_patterns(data)

  0%|          | 0/72371 [00:00<?, ?it/s]

In [59]:
for i,row in summarize_alliance(data, '7ec74bb11b57972a7762381296a9da1c8405d4aa8d8d73bf83c5190042012862').iterrows():
    print(row.type, row.content, row.risk)

  0%|          | 0/1 [00:00<?, ?it/s]

User: 4c2b4a641f57a726f522 | 0.0$/361.61$ | 1.5h 844.69h | 41.0lvl | since 2088 days | risk mean 2.57
User: e3afcee2388b8592c84d | 0.0$/2051.29$ | 2.43h 1838.83h | 50.0lvl | since 2489 days | risk mean 1.5
User: b22c0837be9760bfe888 | 0.0$/839.15$ | 2.62h 395.26h | 41.0lvl | since 1509 days | risk mean 2.0
User: 38cb1a1d58cd05ce24ca | 0.0$/0.0$ | 0.38h 213.72h | 34.0lvl | since 552 days | risk mean 1.0
Action|4c2 join nan
Action|e3a join nan
Action|b22 join nan
Chat|220 speaker eng dude 1.0
Chat|220 we font speak idiot 3.0
Chat|4c2 Fuck you! 6.0
Chat|220 why are you gay? 3.0
Chat|220 this game is fucked up by g@y ppl like u 6.0
Chat|220 bruh get brain dickhead 6.0
Chat|024 Play bro 2.0
Chat|024 U Chinese sucker 3.0
Chat|220 English we dont speak gey 3.0
Chat|220 no i don play with ghey 3.0
Chat|220 wow 1.0
Chat|220 wow 1.0
Chat|220 big skills 1.0
Chat|024 Bro 1.0
Chat|024 U lost 1.0
Chat|4c2 play 2.0
Chat|4c2 I'm Pro Gamer 2.0
Chat|4c2 come on 2.0
Chat|220 yeah pro gamer 2.0
Chat|4c2 c

/home/tobi/development/start_hack_2023/notebooks/../utils/protocol_creation.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alliance_messages['content'] = alliance_messages['message_raw']
/home/tobi/development/start_hack_2023/notebooks/../utils/protocol_creation.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alliance_messages['type'] = "Chat|" + alliance_messages['account_id'].str[:3]


In [107]:
found_patterns_df = pd.DataFrame({'found_patterns':found_patterns.values()}, index=found_patterns.keys())

In [146]:
count = 0
for k,v in found_patterns.items():
    li = [item for sublist in v.values() for item in sublist]
    if 'join_kick_join_kick' in li:
        count+=1
        print(k)
    

08c7469dfa76b9204b53b4dbe21266f241a905d31d013931662cac94463700ad
36176c05057ffe10e957648c9d769f0adc8f9b0872951e108059ccfd30eb8af1
366086fd46e97392286c2997e827d8474cdaa1e61001dbe796b3c161e1c89b17
46df7090c8d12ea2e9b9445803fccdbb51cfacdd5d25238ce919441fed095399
4b5188b322b823a832ca3161649155acd83b40e2fc9c305e648e194d64fa41a4
588766134ff8471a581afd8fbdb981305891f170e55572b39a72e2c265287375
6d517d33ac628a95534d4cde0e17a77a5731c88cbd3f9b5e12357a6cbadee23f
8c726c8df5eec2a2dcf8aa7e13cb01518f383c6742287ab3709632a42b2a97a8
936e49d64821f037c82c3eae0f34aa86cc8e9e66e30294875bc87cc0b40ca559
98fa1018e53b6197edc84bab4599ee78d6ca4f6b03cd9a80ebec392bcf56ba08
a5ff57a54742bbd84f597918271614397b898a99bfe821a66dbe19e4be337439
bc4c1c692638efa4a9ff139e82e87e9d79d247816350840197d3f0a7ef6d1ba7
c7827f1be1e9bcdba403a0cce8486cbceb3de715d9554231e7cab4b790377a3d
ea7c21c4393133450f5f0613acf9a03bdacb679da45f21bb6b42e0d60e406af9


In [183]:
for i,row in summarize_alliance(data, '08c7469dfa76b9204b53b4dbe21266f241a905d31d013931662cac94463700ad').iterrows():
    print(row.type, row.content, row.risk)

  0%|          | 0/1 [00:00<?, ?it/s]

User: 9b80ef78254b7f5eabe8 | 0.0$/172.51$ | 4.47h 562.31h | 44.0lvl | since 1090 days | risk mean 1.0
Chat|a14 Why? 1.0
Chat|a14 JW#1004 2.0
Chat|a14 Why? 1.0
Action|9b8 kicked_out nan
Action|9b8 join nan
Action|9b8 kicked_out nan
Action|9b8 join nan
Action|9b8 kicked_out nan
Action|9b8 join nan
Chat|9b8 arm 1.0
Action|9b8 kicked_out nan
Action|9b8 join nan
Action|9b8 kicked_out nan
Action|9b8 join nan


/home/tobi/development/start_hack_2023/notebooks/../utils/protocol_creation.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alliance_messages['content'] = alliance_messages['message_raw']
/home/tobi/development/start_hack_2023/notebooks/../utils/protocol_creation.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alliance_messages['type'] = "Chat|" + alliance_messages['account_id'].str[:3]


In [108]:
output['found_patterns'] = found_patterns
output['number_patterns_found'] = found_patterns_df.found_patterns.map(lambda inp: len([item for sublist in inp.values() for item in sublist]))

In [218]:
filtered_out = output[((output.current_size >= 30) & (output.number_patterns_found > 1) & (output.risk > 2)) | output.index.isin(['08c7469dfa76b9204b53b4dbe21266f241a905d31d013931662cac94463700ad',
'36176c05057ffe10e957648c9d769f0adc8f9b0872951e108059ccfd30eb8af1',
'366086fd46e97392286c2997e827d8474cdaa1e61001dbe796b3c161e1c89b17'])]

In [219]:
filtered_out['summary'] = 'Not Loaded Because of Limited ChatGPT Budget :('
for i,row in filtered_out.iterrows():
    filtered_out.loc[i,'summary'] =  CHAT_GPT.get(i,'')

/tmp/ipykernel_122861/921256359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_out['summary'] = 'Not Loaded Because of Limited ChatGPT Budget :('


In [222]:
filtered_out.to_json('../data/export/data.json', orient='index')

In [145]:
output.to_json('../data/export/data_full.json', orient='index')

In [ ]:
output.HATE

In [223]:
filtered_out

,current_size,protocol,alliance_account_summaries,risk,HATE_SPEECH,RELIGIOUS,WEBSITE,CHILD_GROOMING,PUBLIC_THREAT,EXTREMISM,SUBVERSIVE,SENTIMENT,POLITICS,filtered,found_patterns,number_patterns_found,summary
alliance_id,,,,,,,,,,,,,,,,,
08c7469dfa76b9204b53b4dbe21266f241a905d31d013931662cac94463700ad,48,{223564: {'author': 'a14bffcfa41c4796b780c4438...,{'9b80ef78254b7f5eabe82a1f7e3f6ceca704acae3f7a...,1.250000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.0,0.000000,{'9b80ef78254b7f5eabe82a1f7e3f6ceca704acae3f7a...,1.0,
15a1b0e15ebdfeb81100492b11d57b22864d8c33519dec5b50b33f8986546824,49,{389301: {'author': 'b4bebd6528fea257c32a58c4f...,{'21c5108a4fe7fbac6994b59a954c42c313eed60ee21f...,2.002326,-1.000000,-0.965116,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.0,0.006977,{'950e08112e692ca248f4e07f2a5711b00f72a3d92814...,2.0,
1762ea9e36fa3852723960be20c9f5f9f98dff1d23343b72f82e715371aaaff9,46,{37026: {'author': 'd0bffba8b1aaa3bf0b13b0a7ff...,{'37d64b42f33730ae54087128883e07be2320b8c87916...,2.352941,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.0,0.058824,{'37d64b42f33730ae54087128883e07be2320b8c87916...,2.0,The group chat in the mobile game Clash Royal ...
1e157931dc27e5065ebee730e437515afe7bd72a0200486ecd60e0efc35c6f13,49,{372051: {'author': '7bf559e1be1a7f30e66cf01eb...,{'0c10b4000da859ac4dd93832ef516ab3d2a21cd7566f...,2.777778,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.0,0.000000,{'0c10b4000da859ac4dd93832ef516ab3d2a21cd7566f...,2.0,
36176c05057ffe10e957648c9d769f0adc8f9b0872951e108059ccfd30eb8af1,40,{371785: {'author': '92274245253708ebe78620a69...,{'609f0a413073c40ec8f897b82b789f2fed6c5650bbdd...,1.666667,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.0,0.000000,{'609f0a413073c40ec8f897b82b789f2fed6c5650bbdd...,2.0,
366086fd46e97392286c2997e827d8474cdaa1e61001dbe796b3c161e1c89b17,0,{398076: {'author': '8b114cef87055e78142cd33cd...,{'488ef7ebbc031ec541dbbe0797dbc257ec2d7c41448b...,2.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.0,0.000000,{'8b114cef87055e78142cd33cd08f2b3845433d32b080...,3.0,
4393dc7e6e71148e23ee789fc38b9402ac39b2b6ad659dc084e291cc12b69de6,43,{408614: {'author': 'de9716f1723476699527383b0...,{'0a8f6cecee1fd014cae936fc33394397f9ea91d4d41f...,2.600000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.0,0.000000,{'de9716f1723476699527383b01fd9a9bef2e98e20e29...,2.0,
51a9c8d4d716e10cf08d806e93c37dd0c17b840f5a26dd5a76a7108ee19bc54f,30,{191396: {'author': '99e9c551290e3e245c959efbd...,{'01f6d43c8d17ef4c5b9385239a4e6ef7ca4e76d32d4d...,2.692308,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.0,0.076923,{'f77903d97ad5137ab552abd361bb00245c6771bda72d...,2.0,
5431064c4aa48933423636fe78a9d22e6f390904fea56f54f1f6b1c4f8b13ae9,50,{373564: {'author': '6a4d9f2df1e07fcefa1fa281d...,{'15385986ff1f2a3c9e1bd33b48c2419d1d76c4468b03...,2.541667,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.0,0.041667,{'6a4d9f2df1e07fcefa1fa281d61c33e918d67d97c50d...,2.0,


In [134]:
alliance_id = filtered_out.index[5]
print(alliance_id)
summary = summarize_alliance(data, alliance_id)
print(create_chatgpt_prompt(summary))

5431064c4aa48933423636fe78a9d22e6f390904fea56f54f1f6b1c4f8b13ae9


  0%|          | 0/1 [00:00<?, ?it/s]

User: 6a4d9f2df1e07fcefa1f | 4.99$/340.12$ | 2.02h 1305.94h | 50.0lvl | since 1393 days | risk mean 2.5
User: f2ac2b941663885d511c | 0.0$/34.93$ | 4.04h 372.1h | 39.0lvl | since 1323 days | risk mean 4.71
User: 15385986ff1f2a3c9e1b | 0.0$/0.0$ | 0.03h 39.42h | 1.0lvl | since 1097 days | risk mean 2.0
User: e6ddddc2a55d0d1080f3 | 0.0$/168.63$ | 2.04h 1203.6h | 46.0lvl | since 2469 days | risk mean 2.0
Summarize the following group chat from the mobile game clash royal:

User 6a4 joins
User 6a4 says: loser
User 6a4 leaves
User f2a joins
User f2a says: Fk loser
User f2a says: You
User f2a says: Loser
User f2a leaves
User f2a joins
User f2a says: Fk you
User f2a says: Bitch
User f2a leaves
User e6d leaves
User e6d joins
User af9 says: welcome back Soulja and asapkeon
User af9 says: ignore those clowns
User e6d says: They just mad lol
User af9 says: my bad
User af9 says: lemme try a better deck
User af9 says: lol I suck
User af9 says: I deleted my 2v2 deck
User af9 says: big mistake
User 64

/home/tobi/development/start_hack_2023/notebooks/../utils/protocol_creation.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alliance_messages['content'] = alliance_messages['message_raw']
/home/tobi/development/start_hack_2023/notebooks/../utils/protocol_creation.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alliance_messages['type'] = "Chat|" + alliance_messages['account_id'].str[:3]


In [149]:
filtered_out.shape

(31, 16)

In [129]:
CHAT_GPT = {
    '1762ea9e36fa3852723960be20c9f5f9f98dff1d23343b72f82e715371aaaff9':'The group chat in the mobile game Clash Royal started with User d0b challenging someone to prove something. User 656 took up the challenge, but User d0b warned him not to act like a "dikkkhed." User 656 was later kicked out of the chat. User d0b then talked about different decks in the game, emphasizing that matchups and counters are essential to winning. User 37d briefly joined the chat and made a negative comment about the clan before leaving. The conversation then shifted to upgrading cards and discussing the 7x mode of the game. User 8a1 expressed relief about not being present during a previous incident and talked about their Princess card.'
    '1e157931dc27e5065ebee730e437515afe7bd72a0200486ecd60e0efc35c6f13':'In this group chat from the mobile game Clash Royale, User 7bf joined and insulted someone for BMing, losing, and being a bot. They also challenged someone to fight and then left. User fa8 said that their internet disconnected, and User 0c1 joined briefly to comment on the clan chat and then left.',
    '4393dc7e6e71148e23ee789fc38b9402ac39b2b6ad659dc084e291cc12b69de6':'In this group chat from Clash Royale, User de9 joined and insulted someone to play a real game. User cd7 thanked everyone and left, followed by User aa9 suggesting a deck to try. User 0a8 joined briefly to call someone trash before leaving, and User aa9 commented "no hard feelings" afterwards.
    '51a9c8d4d716e10cf08d806e93c37dd0c17b840f5a26dd5a76a7108ee19bc54f':'In this group chat from the mobile game Clash Royale, User 99e said "Too ez". User f77 joined and instructed Blue Boi to do something. User 01f declared themselves as an elder and threatened to kick others out. User f77 left and User 349 joined briefly to make an inappropriate comment before leaving. User f77 rejoined and User 95a insulted them and expressed confusion. User f77 claimed to be the person they were talking about.',
    '5431064c4aa48933423636fe78a9d22e6f390904fea56f54f1f6b1c4f8b13ae9':'In this group chat from the mobile game Clash Royale, several users join and insult others before leaving. User af9 welcomes back Soulja and asapkeon and apologizes for the behavior of the other users. They then discuss decks and gameplay. User 643 admits to trolling, and User 153 complains about their trophy level and suggests kicking out Pollo.'
    '':'',
    '':'',
    '':'',
    '':'',
    '':'',
    '':'',
    '':'',
    '':'',
    '':'',
    '':'',
}